<a href="https://colab.research.google.com/github/shynisharaf3126/computer-vision-hands-on/blob/main/Detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import joblib
from skimage.feature import hog
from skimage import color
from skimage.transform import pyramid_gaussian
from imutils.object_detection import non_max_suppression
from scipy.special import expit
import cv2

In [ ]:
model = joblib.load('svm_model.npy')

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

scale = 0
detections = []
img = cv2.imread('C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\detection\\inputs\\1\\313.png')

img = cv2.resize(img,(300,200))

(winW, winH) = (64,128)  # size of the sliding window
windowSize = (winW,winH)
downscale = 1.5

In [ ]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

In [ ]:
for resized in pyramid_gaussian(img, downscale=downscale): # loop over each layer of the image that you take
    # loop over the sliding window for each layer of the pyramid
    for (x,y,window_) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
        # if the window does not meet our desired window size, ignore it!
        if window_.shape[0] != winH or window_.shape[1] !=winW or window_.shape[2]!=3: # ensure the sliding window has met the minimum size requirement
            continue
        window = color.rgb2gray(window_)
        #window = cv2.cvtColor(window_, cv2.COLOR_BGR2GRAY) #.rgb2gray(window_[:,:,:3])
        fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')  # extract HOG features from the window captured
        fds = fds.reshape(1, -1) # re shape the image to make a silouhette of hog
        pred = model.predict(fds) # use the SVM model to make a prediction on the HOG features extracted from the window
        
        if pred == 1:
            if model.decision_function(fds) > 1.5:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                print("Detection:: Location -> ({}, {})".format(x, y))
                print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                   int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
                                      int(windowSize[1]*(downscale**scale))))
    scale+=1

Detection:: Location -> (70, 20)
Scale ->  0 | Confidence Score [1.54646229] 

Detection:: Location -> (130, 30)
Scale ->  0 | Confidence Score [1.55580004] 

Detection:: Location -> (110, 40)
Scale ->  0 | Confidence Score [1.50387263] 

Detection:: Location -> (130, 40)
Scale ->  0 | Confidence Score [1.73159911] 

Detection:: Location -> (0, 50)
Scale ->  0 | Confidence Score [1.62724782] 



In [ ]:
'''
rects = []
scores = []
for (x, y, dec_func, w, h) in detections:
    rects.append([x, y, x + w, y + h])
    score = expit(dec_func)
    scores.append(score)
rects = np.array(rects)
scores = np.array(scores)
print(rects)
pick = non_max_suppression(rects, probs=scores, overlapThresh=0.3)
'''

[[ 70  20 134 148]
 [130  30 194 158]
 [110  40 174 168]
 [130  40 194 168]
 [  0  50  64 178]]


In [ ]:
pick

array([[ 70,  20, 134, 148]])

In [ ]:
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
sc = [score[0] for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh=0.3)

detection confidence score:  [1.5464622850414484, 1.5558000424677132, 1.5038726275539658, 1.7315991083751636, 1.6272478214932675]


In [ ]:
pick

array([[130,  40, 194, 168],
       [  0,  50,  64, 178],
       [ 70,  20, 134, 148]])

In [ ]:
'''
for (xA, yA, xB, yB) in pick:
    for (xA_, yA_, xB_, yB_), score in zip(rects, sc):
        if (xA, yA, xB, yB) == rect:
            print(score)
    #cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
    #cv2.putText(img,'Human',(xA-2,yA-2),1,0.75,(255,255,0),1)
'''

"\nfor (xA, yA, xB, yB) in pick:\n    for (xA_, yA_, xB_, yB_), score in zip(rects, sc):\n        if (xA, yA, xB, yB) == rect:\n            print(score)\n    #cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)\n    #cv2.putText(img,'Human',(xA-2,yA-2),1,0.75,(255,255,0),1)\n"

In [ ]:
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
    cv2.putText(img,'Human',(xA-2,yA-2),1,0.75,(255,255,0),1)

In [ ]:
cv2.imshow('Detection', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# video

video = cv2.VideoCapture(0)
while True: 
    check, frame = video.read()

video.release()
cv2.destroyAllWindows() 